In [63]:

from ipynb.fs.full.utils import resetRandom

resetRandom()

import glob
import h5py
import tqdm
import os
from tensorflow.keras import Input
from tensorflow.keras.preprocessing import image
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.applications.inception_v3 import InceptionV3, preprocess_input as preprocess_Inception
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input as preprocess_VGG16
from keras_facenet import FaceNet
from ipynb.fs.full.utils import CLASS_NAMES, makedirs

In [64]:
def Facenet_model():
    model = FaceNet()
    return model


In [57]:
def Facenet_feature_extract(image_path, model, cls, mode):
    feat_aug = []
    class_name = []
    img = image.load_img(image_path, target_size=(224,224,3))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    if mode == 'train':
        datagen = ImageDataGenerator(
            rescale=1/255,      #rescaling values by dividing each pixel by 255
            rotation_range=10,  # randomly rotate images by 10 degrees
            width_shift_range=0.1,  # randomly shift images horizontally by 10% of width
            height_shift_range=0.1,  # randomly shift images vertically by 10% of height
            zoom_range=0.1,  # randomly zoom images by up to 10%
            brightness_range=[0.9, 1.1],  # randomly adjust brightness by up to 10%
            shear_range=0.1,  # randomly apply shear transformation by up to 10%
            horizontal_flip=True,  # randomly flip images horizontally
            fill_mode='nearest',  # fill any pixels lost during augmentation with the nearest pixel value 
        )
    else:
        datagen = ImageDataGenerator(rescale=1/255)
        
    aug_iter = datagen.flow(x)

    for i in range(30):  # extract features from 30 augmented images
        batch = next(aug_iter)
        feature = model.embeddings(x)
        flat = feature.flatten()
        feat_aug.append(flat)
        
    return list(np.array(feat_aug).flatten()), CLASS_NAMES.index(cls)

<h3>Extracting features for train data</h3>

In [59]:
data_save_path = 'features/'
data_path = 'train/'
Facenet_model = Facenet_model()
makedirs(data_save_path)
feats_dict = {'Facenet_features': []}
labels_dict = {'Facenet_labels': []}
print(CLASS_NAMES)
for cls in CLASS_NAMES:
    if cls not in '.DS_Store':
        print('[INFO] Extracting Features For Class :: {0}'.format(cls))
        for img_path in tqdm.tqdm(glob.glob(os.path.join(data_path, cls, '*.jpg'))):
            features, labels = Facenet_feature_extract(img_path, Facenet_model, cls,'train')
            
            
            feats_dict['Facenet_features'].append(features)
            labels_dict['Facenet_labels'].append(labels)


for k, v in feats_dict.items():
    h5f_data = h5py.File(os.path.join(data_save_path, '{0}.h5'.format(k+"_train")), 'w')
    h5f_data.create_dataset('features', data=np.array(v))
    h5f_data.close()

for k, v in labels_dict.items():
    h5f_data = h5py.File(os.path.join(data_save_path, '{0}.h5'.format(k+"_train")), 'w')
    h5f_data.create_dataset('labels', data=np.array(v))
    h5f_data.close()

['nagarjuna_akkineni', 'madhavan', 'kamal_haasan', 'saif_ali_khan', 'mohanlal', 'prabhas', 'sanjay_dutt', 'akshay_kumar', 'ramya_krishnan', 'salman_khan']
[INFO] Extracting Features For Class :: nagarjuna_akkineni


  0%|                                                                                               | 0/45 [00:00<?, ?it/s]

1/1 [==============================] - 0s 52ms/step


  2%|█▉                                                                                     | 1/45 [00:02<02:01,  2.77s/it]

1/1 [==============================] - 0s 51ms/step


...

 96%|██████████████████████████████████████████████████████████████████████████████████▏   | 43/45 [01:38<00:04,  2.31s/it]

1/1 [==============================] - 0s 56ms/step


 98%|████████████████████████████████████████████████████████████████████████████████████  | 44/45 [01:40<00:02,  2.30s/it]

1/1 [==============================] - 0s 55ms/step


100%|██████████████████████████████████████████████████████████████████████████████████████| 45/45 [01:42<00:00,  2.29s/it]


<h3>Extracting features for test data</h3>

In [62]:
data_save_path = 'features/'
data_path = 'test/'
Facenet_model = Facenet_model()
makedirs(data_save_path)
feats_dict = {'Facenet_features': []}
labels_dict = {'Facenet_labels': []}
print(CLASS_NAMES)
for cls in CLASS_NAMES:
    if cls not in '.DS_Store':
        print('[INFO] Extracting Features For Class :: {0}'.format(cls))
        for img_path in tqdm.tqdm(glob.glob(os.path.join(data_path, cls, '*.jpg'))):
            features, labels = Facenet_feature_extract(img_path, Facenet_model, cls,'test')

            feats_dict['Facenet_features'].append(features)
            labels_dict['Facenet_labels'].append(labels)



for k, v in feats_dict.items():
    h5f_data = h5py.File(os.path.join(data_save_path, '{0}.h5'.format(k+"_test")), 'w')
    h5f_data.create_dataset('features', data=np.array(v))
    h5f_data.close()

for k, v in labels_dict.items():
    h5f_data = h5py.File(os.path.join(data_save_path, '{0}.h5'.format(k+"_test")), 'w')
    h5f_data.create_dataset('labels', data=np.array(v))
    h5f_data.close()

['nagarjuna_akkineni', 'madhavan', 'kamal_haasan', 'saif_ali_khan', 'mohanlal', 'prabhas', 'sanjay_dutt', 'akshay_kumar', 'ramya_krishnan', 'salman_khan']
[INFO] Extracting Features For Class :: nagarjuna_akkineni


  0%|                                                                                                | 0/5 [00:00<?, ?it/s]

1/1 [==============================] - 0s 53ms/step


 20%|█████████████████▌                                                                      | 1/5 [00:02<00:10,  2.73s/it]

1/1 [==============================] - 0s 54ms/step


...

 60%|████████████████████████████████████████████████████▊                                   | 3/5 [00:06<00:04,  2.11s/it]

1/1 [==============================] - 0s 57ms/step


 80%|██████████████████████████████████████████████████████████████████████▍                 | 4/5 [00:08<00:02,  2.11s/it]

1/1 [==============================] - 0s 57ms/step


100%|████████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:10<00:00,  2.12s/it]
